****************************************************************

In [1]:
# Importing the root of this bootcamp
import os.path as osp
import sys

sys.path.append(osp.abspath('..'))

# PyTorch Lightning abstraction basics

Putting it all together with PL abstraction mechanics

Let's first load all the necessary params

In [2]:
import numpy as np
import os
import os.path as osp
import pytorch_lightning as pl

import config

# Ensures this Notebook's reproducibility
pl.seed_everything(42, workers=True)

step = config.config['timestep']
params = config.params[str(step)]['flattened']

Global seed set to 42


## Model and training logic

In [3]:
!cat models.py

# MIT License
# 
# Copyright (c) 2022 alxyok
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, A

In [4]:
import models

In [5]:
x_feats = params['x_shape'][-1]
y_feats = params['y_shape'][-1]

In [6]:
print(f'x number of features: {x_feats}')
print(f'y number of features: {y_feats}')

x number of features: 4128
y number of features: 552


In [7]:
mlp = models.LitMLP(
    in_channels=x_feats,
    hidden_channels=100,
    out_channels=y_feats
)
mlp

LitMLP(
  (normalization_layer): Normalize()
  (net): Sequential(
    (0): Normalize()
    (1): Linear(in_features=4128, out_features=100, bias=True)
    (2): SiLU()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): SiLU()
    (5): Linear(in_features=100, out_features=100, bias=True)
    (6): SiLU()
    (7): Linear(in_features=100, out_features=552, bias=True)
  )
)

## Dataset creation and data loading mechanics

In [8]:
!cat data.py

# MIT License
# 
# Copyright (c) 2022 alxyok
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, A

In [9]:
import data

* `batch_size` sets the number of element in a batch of data.
* `num_workers` sets the number of workers the DataLoader can spawn to handle data loading and Dataset batching.

In [ ]:
import psutil
cores = psutil.cpu_count(logical=False)

In [3]:
datamodule = data.FlattenedDataModule(
    batch_size=1024,
    
    # In CPU-only setup, make sure you have enough cores to handle the job
    # Otherwise this option will start a serious bottleneck
    num_workers=cores // 2
)

NameError: name 'data' is not defined

## Orchestrating the training

In [28]:
logger = pl.loggers.tensorboard.TensorBoardLogger(
    save_dir=config.logs_path,
    name='flattened_mlp_logs',
    log_graph=True
)

All the training instrumentation is done by an object call the Trainer. You can fix parameters such as:
* `max_epochs` unless an early stopping happens
* `accelerator` type and `device` logical number

Notably interesting: 
* `callbacks` to handle in-betweens
* `gradient_clip_val` and `gradient_clip_algorithm` to setup the gradient clipping
* `logger` to interface with loss and metrics logging
* `resume_from_checkpoint` helps resuming a previously initiated training
* `amp_backend` to switch to Nvidia Apex framework for Automatic Mixed Precision support

In [29]:
cpu_trainer = pl.Trainer(
    max_epochs=1,
    logger=logger,
    deterministic=True,
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Training CPU is a one-line

In [30]:
cpu_trainer.fit(model=mlp, datamodule=datamodule)


  | Name                | Type       | Params
---------------------------------------------------
0 | normalization_layer | Normalize  | 0     
1 | net                 | Sequential | 488 K 
---------------------------------------------------
488 K     Trainable params
0         Non-trainable params
488 K     Total params
1.955     Total estimated model params size (MB)


Global seed set to 42


Epoch 0:   7%|▋         | 63/954 [01:56<27:24,  1.85s/it, loss=0.948, v_num=6, train_loss=1.190]  

Never forget to test. The handy thing with the `Trainer` is, if a `.test()` is called somewhere at runtime, once a `SIGTERM` is thrown by the runtime such as a `KeyboardInterruptError`, it gets caught by Lightning, which tries to then run the test anyway.

## Data Parallelism

A Note on GPU training. We naturally assume that GPU is better than CPU

Now let's go single-node multi-GPU. The same model will be pushed to all available devices, each of which will
1. Perform forward pass with its specific batch of data
2. Compute the loss and perform backward pass including weights update
4. The weights are then collected are synchronized across all devices for next pass

#### DDP Strategy

In [1]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [31]:
gpu_trainer = pl.Trainer(
    accelerator='gpu',
    devices=-1,
    max_epochs=1,
    logger=logger,
    deterministic=True,
    # amp_backend='apex'
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [32]:
gpu_trainer.fit(model=mlp, datamodule=datamodule)
gpu_trainer.test(model=mlp, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Global seed set to 42
initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2


Epoch 0:   7%|▋         | 63/954 [02:12<31:17,  2.11s/it, loss=0.948, v_num=6, train_loss=1.190]

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/multiprocessing/spawn.py", line 59, in _wrap
    fn(i, *args)
  File "/opt/conda/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py", line 207, in _wrapped_function
    self._worker_setup(process_idx)
  File "/opt/conda/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py", line 217, in _worker_setup
    self.cluster_environment, self.torch_distributed_backend, self.global_rank, self.world_size
  File "/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py", line 388, in init_dist_connection
    torch_distributed_backend, rank=global_rank, world_size=world_size, **kwargs
  File "/opt/conda/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py", line 576, in init_process_group
    store, rank, world_size = next(rendezvous_iterator

#### Horovod

With a simple change in the Trainer options, you can rely on horovod backend to perform the computations. Prallelism is achieved by SPMD with MPI: one process per GPU, and collective computing is made by process of rank 0.

## Model Parallelism

You should really go for model parallelism starting at 500M parameters. No material on that, just know that it exists and it is complex. Lightning comes standard with a series of distrubtion strategies, each with a specific implementation related to the network that first introduced it.

Refer to the Doc for more info.